# Análisis de Series Temporales

## Visualización e Intuición

La herramienta principal para el análisis de series temporales es la inspección visual, para ello resulta imprescindible manejar herramientas gráficas. En este curso utilizaremos la librería `matplotlib`. Para leer archivos lo haremos utilizando la librería `csv`.

Para abrir un archivo utilzaremos la sentencia `open` con el handler `with`.

In [1]:
import csv
import matplotlib.pyplot as plt

Siempre hay que conocer la forma que tiene el archivo a leer. En nuestro caso examinaremos dos de ellos:
1. Precios de mercado de Google
```
Date,Open,High,Low,Close,Adj Close,Volume
2010-01-04,15.689439,15.753504,15.621622,15.684434,15.684434,78169752
2010-01-05,15.695195,15.711712,15.554054,15.615365,15.615365,120067812
2010-01-06,15.662162,15.662162,15.174174,15.221722,15.221722,158988852
2010-01-07,15.250250,15.265265,14.831081,14.867367,14.867367,256315428
2010-01-08,14.814815,15.096346,14.742492,15.065566,15.065566,188783028
```
2. Datos del SMN 
```
FECHA     HORA  TEMP   HUM   PNM    DD    FF     NOMBRE                                             
         [HOA]  [�C]   [%]  [hPa]  [gr] [km/hr]                                                     
02062023     0  18.0   84  1016.4   50   11     AEROPARQUE AERO                                     
02062023     1  18.4   83  1015.5   20    9     AEROPARQUE AERO                                     
02062023     2  18.3   83  1015.2   20    9     AEROPARQUE AERO                                     
02062023     3  18.4   83  1015.2  360   13     AEROPARQUE AERO                                     
02062023     4  18.2   84  1014.6  350    9     AEROPARQUE AERO                                     
```

In [2]:
data_google = '../Data/googleStock/Google_Stock_Train (2010-2022).csv'
data_smn = '../Data/junio-SMN/horario/datohorario20230602.txt'

In [62]:
with open( data_google, 'r') as fp:
    reader = csv.DictReader(fp, delimiter=',')  # Leer como diccionarios
    g_data = []                                 # Lista de diccionarios
    for row in reader:
        g_data.append(row)
for k,v in g_data[1].items():
    print(f'{k}: {v}')

Date: 2010-01-05
Open: 15.695195
High: 15.711712
Low: 15.554054
Close: 15.615365
Adj Close: 15.615365
Volume: 120067812


In [66]:
# Los datos del SMN tienen dos columnas de encabezado, hay que leer con cuidado, 
# parece estar hecho en un encoding traido de windows.
# Los archivos parecen tener ancho de columnas fijas
with open(data_smn, 'rb') as fp:
    headers = next(fp).decode().split()
    units = next(fp).decode(errors='replace').split()
    smn_data = []
    for line in fp:
        lines = line.decode(errors='replace')
        smn_data.append([
            lines[:8].strip(), 
            lines[8:14].strip(), 
            lines[14:20].strip(), 
            lines[20:25].strip(), 
            lines[25:33].strip(), 
            lines[33:38].strip(),
            lines[38:43].strip(),
            lines[43:].strip()
            ])

# limpiando las columnas con salto de carro
clean = []
for l in smn_data:
    if l[0] != '':
        clean.append(l)

for l in range(len(clean)):
    for it in range(len(clean[l])):
        if clean[l][it] == '':
            clean[l][it] = '-1'


smn_data = clean
print(headers)
print(units)
import random
for i in range(50):
    print(random.choice(smn_data))

['FECHA', 'HORA', 'TEMP', 'HUM', 'PNM', 'DD', 'FF', 'NOMBRE']
['[HOA]', '[�C]', '[%]', '[hPa]', '[gr]', '[km/hr]']
['02062023', '6', '19.0', '84', '1011.5', '30', '17', 'ROSARIO AERO']
['02062023', '11', '11.2', '49', '994.8', '320', '33', 'PUERTO DESEADO AERO']
['02062023', '12', '10.2', '69', '1448.0', '250', '13', 'MALARGUE AERO']
['02062023', '18', '26.4', '63', '1004.1', '0', '0', 'ORAN AERO']
['02062023', '22', '20.3', '81', '1012.2', '360', '9', 'LA PLATA AERO']
['02062023', '17', '23.9', '53', '1005.4', '180', '11', 'SAN LUIS AERO']
['02062023', '15', '27.4', '50', '1003.8', '340', '7', 'CORDOBA OBSERVATORIO']
['02062023', '8', '11.1', '94', '1007.6', '20', '4', 'TUCUMAN AERO']
['02062023', '21', '1.2', '92', '989.2', '320', '17', 'RIO GRANDE B.A.']
['02062023', '5', '-3.2', '32', '3116.0', '0', '0', 'LA QUIACA OBSERVATORIO']
['02062023', '9', '19.5', '85', '1015.9', '70', '20', 'FORMOSA AERO']
['02062023', '8', '15.4', '98', '1014.0', '70', '9', 'PRESIDENCIA ROQUE SAENZ PE']
[

Utilizando la libreria `datetime` para manejar datos de fechas y horas.

En este caso se convertirá el dato de fechas a objetos de datetime

In [67]:
from datetime import datetime

In [68]:
smn_data_standarized = [{}]
for d in smn_data:
    # Since data is not very well structured and a lot of carriage return appear
    if len(d) > 7:
        smn_data_standarized.append({
            'fecha': datetime.strptime(d[0],'%d%m%Y').date(),
            'hora': int(d[1]),
            'temp': float(d[2]),
            'hum': float(d[3]),
            'pres': float(d[4]),
            'v_direcc': int(d[5]),
            'v_intens': float(d[6]),
            'estac': d[7]
        }
        )
    # try:
    #     print(datetime.strptime(d[0],'%d%m%Y').date())
    # except:
    #     print(d[0])